In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q -U "tensorflow-text==2.8.*"
!pip install -q tf-models-official==2.7.0

     |████████████████████████████████| 4.9 MB 8.3 MB/s 
     |████████████████████████████████| 498.0 MB 13 kB/s 
     |████████████████████████████████| 5.8 MB 86.5 MB/s 
     |████████████████████████████████| 462 kB 86.8 MB/s 
     |████████████████████████████████| 1.4 MB 83.7 MB/s 
     |████████████████████████████████| 1.8 MB 8.5 MB/s 
     |████████████████████████████████| 118 kB 84.8 MB/s 
     |████████████████████████████████| 352 kB 83.3 MB/s 
     |████████████████████████████████| 238 kB 87.4 MB/s 
     |████████████████████████████████| 1.1 MB 68.1 MB/s 
     |████████████████████████████████| 1.3 MB 61.0 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 


In [ ]:
import pandas as pd
import os
import shutil
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
import tensorflow_addons as tfa

tf.get_logger().setLevel('ERROR')

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
# Create train set
emotions = ["anger", "anticipation", "fear", "joy", "sadness"]
data_file_train = "/content/drive/MyDrive/ling490/TermProject/data/small.csv"
train_lang = "en"

for e in emotions: os.makedirs(f"/content/train/{train_lang}/{e}")

df = pd.read_csv(data_file_train)
sentences = [[],[],[],[],[]]
for idx,row in df.loc[df['language'] == train_lang].iterrows():
  sentences[emotions.index(row[1])].append(row[0])

for i,e in enumerate(sentences):
  train_sents = e[:int(len(e)*.8)]
  for j,sent in enumerate(train_sents):
    with open(f"/content/train/{train_lang}/{emotions[i]}/{j}.txt", 'w') as f: f.write(sent)

In [ ]:
# Create test set for large languages from file 1
emotions = ["anger", "anticipation", "fear", "joy", "sadness"]
data_file_test = "/content/drive/MyDrive/ling490/TermProject/data/small.csv"
test_langs = ["en", "tl", "zh", "pt"]

for test_lang in test_langs:
  for e in emotions: os.makedirs(f"/content/test/{test_lang}/{e}")

  df = pd.read_csv(data_file_test)
  sentences = [[],[],[],[],[]]
  for idx,row in df.loc[df['language'] == test_lang].iterrows():
    sentences[emotions.index(row[1])].append(row[0])

  for i,sents in enumerate(sentences):
    test_sents = sents[int(len(sents)*.8):]
    for j,sent in enumerate(test_sents):
      with open(f"/content/test/{test_lang}/{emotions[i]}/{j}.txt", 'w') as f: f.write(sent)

In [ ]:
# Create test set for large languages from file 2
emotions = ["anger", "anticipation", "fear", "joy", "sadness"]
data_file_test = "/content/drive/MyDrive/ling490/TermProject/data/low-resource.csv"
test_langs = ["de", "fr", "id", "th", "vi"]

for test_lang in test_langs:
  for e in emotions: os.makedirs(f"/content/test/{test_lang}/{e}")

  df = pd.read_csv(data_file_test)
  sentences = [[],[],[],[],[]]
  for idx,row in df.loc[df['language'] == test_lang].iterrows():
    sentences[emotions.index(row[1])].append(row[0])

  for i,sents in enumerate(sentences):
    test_sents = sents[int(len(sents)*.8):]
    for j,sent in enumerate(test_sents):
      with open(f"/content/test/{test_lang}/{emotions[i]}/{j}.txt", 'w') as f: f.write(sent)

In [ ]:
# Create test set for small languages from file 2
emotions = ["anger", "anticipation", "fear", "joy", "sadness"]
data_file_test = "/content/drive/MyDrive/ling490/TermProject/data/low-resource.csv"
test_langs = ["km", "bn", "my"]

for test_lang in test_langs:
  for e in emotions: os.makedirs(f"/content/test/{test_lang}/{e}")

  df = pd.read_csv(data_file_test)
  sentences = [[],[],[],[],[]]
  for idx,row in df.loc[df['language'] == test_lang].iterrows():
    sentences[emotions.index(row[1])].append(row[0])

  for i,sents in enumerate(sentences):
    for j,sent in enumerate(sents):
      with open(f"/content/test/{test_lang}/{emotions[i]}/{j}.txt", 'w') as f: f.write(sent)

In [ ]:
# Convert to tf datasets
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 1
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    f'/content/train/{train_lang}',
    batch_size=batch_size,
    seed=seed,label_mode='categorical')
train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 2357 files belonging to 5 classes.
Found 953 files belonging to 5 classes.


In [ ]:
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
metrics = [tf.metrics.CategoricalAccuracy(), 
           tfa.metrics.F1Score(num_classes=len(emotions),average='weighted')]

epochs = 12
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 1e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [ ]:
# Load trained model
classifier_model = tf.keras.models.load_model(f"/content/drive/MyDrive/ling490/TermProject/checkpoints/en12")
classifier_model.compile(optimizer, loss, metrics)

In [ ]:
# Evlauate model on each test language
for test_lang in ["en","fr","pt","zh","id","vi","th","bn","de","my","tl","km"]:
  test_ds = tf.keras.utils.text_dataset_from_directory(
    f'/content/test/{test_lang}',
    batch_size=batch_size,label_mode='categorical')
  test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

  loss_m, accuracy, f1 = classifier_model.evaluate(test_ds)
  print(test_lang)
  print(f'Loss: {loss_m}')
  print(f'Accuracy: {accuracy}')
  print(f'F1: {f1}')

Found 590 files belonging to 5 classes.
590/590 [==============================] - 27s 36ms/step - loss: 2.4969 - categorical_accuracy: 0.6017 - f1_score: 0.5992
en
Loss: 2.4969141483306885
Accuracy: 0.6016949415206909
F1: 0.5992365479469299
Found 1314 files belonging to 5 classes.
1314/1314 [==============================] - 46s 35ms/step - loss: 2.6729 - categorical_accuracy: 0.5616 - f1_score: 0.5563
fr
Loss: 2.6729252338409424
Accuracy: 0.5616438388824463
F1: 0.5563379526138306
Found 592 files belonging to 5 classes.
592/592 [==============================] - 21s 36ms/step - loss: 2.6304 - categorical_accuracy: 0.5845 - f1_score: 0.5779
pt
Loss: 2.6303935050964355
Accuracy: 0.5844594836235046
F1: 0.5779013633728027
Found 592 files belonging to 5 classes.
592/592 [==============================] - 25s 43ms/step - loss: 2.0477 - categorical_accuracy: 0.6689 - f1_score: 0.6397
zh
Loss: 2.0476770401000977
Accuracy: 0.6689189076423645
F1: 0.6397345662117004
Found 1243 files belonging to